# Import Libraries

In [1]:
from distutils.log import error
import os, sys, glob, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from corner import corner
from astropy.io import fits
import yaml
from matplotlib import gridspec
from matplotlib.ticker import ScalarFormatter, FuncFormatter
from scipy.stats import pearsonr, gaussian_kde, norm
from itertools import repeat
import IPython.display
from decimal import Decimal

from xspec import *

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

size = 14
plt.rc('font', size=size)
plt.rc("errorbar", capsize=0)
plt.rc('xtick', direction='in', top=True)
plt.rc('ytick', direction='in', right=True)
plt.rc('axes', linewidth=1.25)
#plt.style.use(['dark_background'])
#plt.style.use('seaborn')
#plt.rc('text', usetex=True)
#plt.rc('mathtext', fontset='custom')
#plt.rc('text.latex', preamble=r'\usepackage{newtxtext}\usepackage{newtxmath}')
logformatter = FuncFormatter(lambda y, _: "10$^{{\\text{{{:.2g}}}}}$".format(np.log10(y)))
formatter = FuncFormatter(lambda y, _: "{0}".format(y))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
colors = 30*colors
seaborngreen = '#55A868'
seabornred = '#C44E52'
seabornblue = '#4C72B0'
seabornorange = '#EF843C'
seabornpink = '#E77F67'
seabornbrown = '#8C564B'
seabornpurple = '#9C9C9C'
seabornolive = '#8DD3C7'
seaborncyan = '#7FCDBB'
seabornmagenta = '#C7B9A2'
seabornyellow = '#FDAE61'

#Xset.cosmo = "67.3,,0.685"
#Xset.abund = "wilm"
#Xset.abund = "angr"
Xset.xsect = "vern"
Xset.cosmo = "70, 0, 0.73"
Plot.device = "/null"
#Plot.device = "/svg"
#Plot.device = "/xs"
Plot.background = True
Plot.xAxis = "keV"
Plot.xLog=True
Plot.yLog=True
Fit.statMethod = 'cstat'
Fit.query = 'yes'

 Cross Section Table set to vern:  Verner, Ferland, Korista, and Yakovlev 1996
Default fit statistic is set to: C-Statistic
   This will apply to all current and newly loaded spectra.


# Simple Apec Function

In [2]:
kT = widgets.FloatSlider(value=0.49,min=0.01,max=10,step=0.05, description="Temperature (keV)")#round((10-0.008)/50,7)
Abundanc = widgets.FloatSlider(value=0.3,min=0,max=1,step=0.05, description="Abundance")
Redshift = widgets.FloatSlider(value=0.5458,min=0,max=10,step=0.05, description="Redshift")
norm = widgets.FloatSlider(value=1e-7,min=1e-10,max=1e-6,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
nH = widgets.FloatSlider(value=0.084,min=0,max=0.1,step=0.001, description="Photoelectric Absorpsion",readout=True,readout_format='.5f')
color = widgets.Dropdown(options=[('Green',seaborngreen),('Red',seabornred),('Blue',seabornblue),('Orange',seabornorange),('Pink',seabornpink),('Brown',seabornbrown),('Purple',seabornpurple),('Cyan',seaborncyan),('Magenta',seabornmagenta),('Yellow',seabornyellow)], description='Color')

#ui1 = widgets.HBox([kT, Abundanc, Redshift, norm, color])
#ui2 = widgets.HBox([kT, Abundanc, Redshift, norm, color])

def apec_model(nH, kT, Abundanc, Redshift, norm):
    t1 = Model("phabs(apec)", "t1")
    t1.phabs.nH.values = nH, -1
    t1.apec.kT.values = kT, -1
    t1.apec.Abundanc.values = Abundanc, -1
    t1.apec.Redshift.values = Redshift, -1
    t1.apec.norm.values =  norm, -1
    Xset.chatter=0
    Plot.addCommand("rescale x 0.1 10")
    Plot.addCommand("rescale y 1e-14 1e-6")
    Plot("model t1")
    mx = Plot.x()
    my = Plot.model()
    AllModels.show()
    return mx,my

#test = apec_model(0.5,0.3,0.5,1e-07)
#print(test)

def plot_apec_model(nH, kT, Abundanc, Redshift, norm, color):
    fig, ax = plt.subplots(figsize=(10,6))
    x,y = apec_model(nH, kT, Abundanc, Redshift, norm)
    ax.plot(x, y, color=color, label=f'{kT} keV',linewidth=1)
    #ax.plot(x, y)
    ax.set_xlabel('Energy (keV)')
    ax.set_ylabel(r'counts/s/keV')
    ax.set_xscale("log")
    ax.set_ylim((1e-14,1e-4))
    ax.set_yscale("log")
    ax.set_xlim((0.1,15))
    ax.grid()
    ax.legend()
    plt.show()

#out1 = widgets.interactive_output(plot_apec_model, {'kT': kT, 'Abundanc': Abundanc, 'Redshift': Redshift, 'norm': norm, 'color': color})
#display(out1, ui1)

interactive(plot_apec_model, nH=nH, kT=kT, Abundanc=Abundanc, Redshift=Redshift, norm=norm, color=color)

interactive(children=(FloatSlider(value=0.084, description='Photoelectric Absorpsion', max=0.1, readout_format…

# Compare two Apec Models

In [4]:
kT = widgets.FloatSlider(value=0.50,min=0.01,max=10,step=0.001, description="Temperature (keV)")#round((10-0.008)/50,7)
Abundanc = widgets.FloatSlider(value=0.3,min=0,max=1,step=0.05, description="Abundance")
Redshift = widgets.FloatSlider(value=0.5458,min=0,max=10,step=0.05, description="Redshift")
norm = widgets.FloatSlider(value=1e-7,min=1e-10,max=1e-6,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
nH = widgets.FloatSlider(value=0.084,min=0,max=0.1,step=0.001, description="Photoelectric Absorpsion",readout=True,readout_format='.5f')
color = widgets.Dropdown(options=[('Green',seaborngreen),('Red',seabornred),('Blue',seabornblue),('Orange',seabornorange),('Pink',seabornpink),('Brown',seabornbrown),('Purple',seabornpurple),('Cyan',seaborncyan),('Magenta',seabornmagenta),('Yellow',seabornyellow)], description='Color')
kT2 = widgets.FloatSlider(value=0.50,min=0.01,max=10,step=0.001, description="Temperature (keV)")#round((10-0.008)/50,7)
Abundanc2 = widgets.FloatSlider(value=0.3,min=0,max=1,step=0.05, description="Abundance")
Redshift2 = widgets.FloatSlider(value=0.5458,min=0,max=10,step=0.05, description="Redshift")
norm2 = widgets.FloatSlider(value=1e-7,min=1e-10,max=1e-6,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
nH2 = widgets.FloatSlider(value=0.084,min=0,max=0.1,step=0.001, description="Photoelectric Absorpsion",readout=True,readout_format='.5f')
color2 = widgets.Dropdown(options=[('Green',seaborngreen),('Red',seabornred),('Blue',seabornblue),('Orange',seabornorange),('Pink',seabornpink),('Brown',seabornbrown),('Purple',seabornpurple),('Cyan',seaborncyan),('Magenta',seabornmagenta),('Yellow',seabornyellow)], description='Color')

#ui1 = widgets.HBox([kT, Abundanc, Redshift, norm, color])
#ui2 = widgets.HBox([kT, Abundanc, Redshift, norm, color])

def apec_model(nH,kT, Abundanc, Redshift, norm):
    t1 = Model("apec", "t1")
    t1.phabs.nH.values = nH, -1
    t1.apec.kT.values = kT, -1
    t1.apec.Abundanc.values = Abundanc, -1
    t1.apec.Redshift.values = Redshift, -1
    t1.apec.norm.values =  norm, -1
    Xset.chatter=0
    Plot.addCommand("rescale x 0.1 10")
    Plot.addCommand("rescale y 1e-14 1e-6")
    Plot("model t1")
    mx = Plot.x()
    my = Plot.model()
    AllModels.show()
    return mx,my

#test = apec_model(0.5,0.3,0.5,1e-07)
#print(test)

def plot_2apec_model(nH, kT, Abundanc, Redshift, norm, color, nH2, kT2, Abundanc2, Redshift2, norm2, color2):
    fig, ax = plt.subplots(2,figsize=(10,6))
    x,y = apec_model(nH, kT, Abundanc, Redshift, norm)
    ax[0].plot(x, y, color=color, label=f'{round(kT,5)} keV',linewidth=1)
    x2,y2 = apec_model(nH2, kT2, Abundanc2, Redshift2, norm2)
    ax[0].plot(x2, y2, color=color2, label=f'{round(kT2,5)} keV',linewidth=1)
    ratio = [y/y2 if y2 else 0 for y,y2 in zip(y,y2)]
    #ax.plot(x, y)
    ax[1].set_xlabel('Energy (keV)')
    ax[0].set_ylabel(r'counts/s/keV')
    ax[0].set_xscale("log")
    ax[0].set_ylim((1e-14,1e-4))
    ax[0].set_yscale("log")
    ax[0].set_xlim((0.1,10))
    ax[0].grid()
    ax[0].legend()
    ax[1].plot(x, ratio, color="black", label=f'M1/M2 ratio',linewidth=1)
    ax[1].set_ylabel(r'Ratio')
    ax[1].set_xscale("log")
    #ax[1].set_ylim((0,5))
    ax[1].set_yscale("linear")
    ax[1].set_xlim((0.1,10))
    ax[1].grid()
    ax[1].legend()
    plt.show()

#out1 = widgets.interactive_output(plot_apec_model, {'kT': kT, 'Abundanc': Abundanc, 'Redshift': Redshift, 'norm': norm, 'color': color})
#display(out1, ui1)

interactive(plot_2apec_model, nH=nH, kT=kT, Abundanc=Abundanc, Redshift=Redshift, norm=norm, color=color, nH2=nH2, kT2=kT2, Abundanc2=Abundanc2, Redshift2=Redshift2, norm2=norm2, color2=color2)

interactive(children=(FloatSlider(value=0.5, description='Temperature (keV)', max=10.0, min=0.01, step=0.001),…

# Compare two CXB spectra with Two Apec Components

In [17]:
kT = widgets.FloatSlider(value=9.65754E-02,min=0.01,max=0.2,step=0.001, description="kT1 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm = widgets.FloatSlider(value=5.56817E-07,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
color = widgets.Dropdown(options=[('Green',seaborngreen),('Red',seabornred),('Blue',seabornblue),('Orange',seabornorange),('Pink',seabornpink),('Brown',seabornbrown),('Purple',seabornpurple),('Cyan',seaborncyan),('Magenta',seabornmagenta),('Yellow',seabornyellow)], description='Color')
kT2 = widgets.FloatSlider(value=0.178613,min=0.01,max=0.7,step=0.001, description="kT2 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm2 = widgets.FloatSlider(value=1.64881E-06,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
color2 = widgets.Dropdown(options=[('Green',seaborngreen),('Red',seabornred),('Blue',seabornblue),('Orange',seabornorange),('Pink',seabornpink),('Brown',seabornbrown),('Purple',seabornpurple),('Cyan',seaborncyan),('Magenta',seabornmagenta),('Yellow',seabornyellow)], description='Color')
nH = widgets.FloatSlider(value=0.084,min=0.01,max=0.1,step=0.001, description="Photoelectric Absorpsion",readout=True,readout_format='.5f')
power = widgets.FloatSlider(value=1.41,min=0,max=3,step=0.1, description="PhoIndex",readout=True,readout_format='.5f')
pnorm = widgets.FloatSlider(value=7.80479E-07,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')

kT3 = widgets.FloatSlider(value=9.65754E-02,min=0.01,max=0.2,step=0.001, description="kT1 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm3 = widgets.FloatSlider(value=5.56817E-07,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
color3 = widgets.Dropdown(options=[('Green',seaborngreen),('Red',seabornred),('Blue',seabornblue),('Orange',seabornorange),('Pink',seabornpink),('Brown',seabornbrown),('Purple',seabornpurple),('Cyan',seaborncyan),('Magenta',seabornmagenta),('Yellow',seabornyellow)], description='Color')
kT4 = widgets.FloatSlider(value=0.178613,min=0.01,max=0.7,step=0.001, description="kT2 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm4 = widgets.FloatSlider(value=1.64881E-06,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
color4 = widgets.Dropdown(options=[('Green',seaborngreen),('Red',seabornred),('Blue',seabornblue),('Orange',seabornorange),('Pink',seabornpink),('Brown',seabornbrown),('Purple',seabornpurple),('Cyan',seaborncyan),('Magenta',seabornmagenta),('Yellow',seabornyellow)], description='Color')
nH2 = widgets.FloatSlider(value=0.084,min=0.01,max=0.1,step=0.001, description="Photoelectric Absorpsion",readout=True,readout_format='.5f')
power2 = widgets.FloatSlider(value=1.41,min=0,max=3,step=0.1, description="PhoIndex",readout=True,readout_format='.5f')
pnorm2 = widgets.FloatSlider(value=7.80479E-07,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')

def cxb_model_2apec(kT, norm, nH, kT2, norm2, power, pnorm):
    cxb2 = Model("apec + phabs(apec + powerlaw)", "cxb2")
    cxb2.apec.kT.values = kT, -1
    cxb2.apec.Abundanc.values = 1, -1
    cxb2.apec.Redshift.values = 0, -1
    cxb2.apec.norm.values =  norm, -1
    cxb2.phabs.nH.values = nH, -1
    cxb2.apec_3.kT.values = kT2, -1
    cxb2.apec_3.Abundanc.values = 1, -1
    cxb2.apec_3.Redshift.values = 0, -1
    cxb2.apec_3.norm.values = norm2, -1
    cxb2.powerlaw.PhoIndex.values = power, -1
    cxb2.powerlaw.norm.values = pnorm, -1
    Xset.chatter=0
    Plot.addCommand("rescale x 0.1 10")
    Plot.addCommand("rescale y 1e-14 1e-5")
    Plot("model cxb2")
    mx = Plot.x()
    my = Plot.model()
    AllModels.show()
    return mx,my

def plot_2apec_cxb_models(kT, norm, nH, kT2, norm2, power, pnorm, color, kT3, norm3, nH2, kT4, norm4, power2, pnorm2, color2):
    fig, ax = plt.subplots(2,figsize=(10,6))
    x,y = cxb_model_2apec(kT, norm, nH, kT2, norm2, power, pnorm)
    ax[0].plot(x, y, color=color, label=f'CXB1',linewidth=1)
    x2,y2 = cxb_model_2apec(kT3, norm3, nH2, kT4, norm4, power2, pnorm2)
    ax[0].plot(x2, y2, color=color2, label=f'CXB2',linewidth=1)
    ratio = [y/y2 if y2 else 0 for y,y2 in zip(y,y2)]
    #ax.plot(x, y)
    ax[1].set_xlabel('Energy (keV)')
    ax[0].set_ylabel(r'counts/s/keV')
    ax[0].set_xscale("log")
    ax[0].set_ylim((1e-10,1e-2))
    ax[0].set_yscale("log")
    #ax[0].set_xlim((0.5,3))
    ax[0].grid()
    ax[0].legend()
    ax[1].plot(x, ratio, color="black", label=f'M1/M2 ratio',linewidth=1)
    ax[1].set_ylabel(r'Ratio')
    ax[1].set_xscale("log")
    #ax[1].set_ylim((0,5))
    ax[1].set_yscale("linear")
    #ax[1].set_xlim((0.5,3))
    ax[1].grid()
    ax[1].legend()
    plt.show()

#out1 = widgets.interactive_output(plot_apec_model, {'kT': kT, 'Abundanc': Abundanc, 'Redshift': Redshift, 'norm': norm, 'color': color})
#display(out1, ui1)

interactive(plot_2apec_cxb_models, kT=kT, norm=norm, nH=nH, kT2=kT2, norm2=norm2, power=power, pnorm=pnorm, color=color, kT3=kT3, norm3=norm3, nH2=nH2, kT4=kT4, norm4=norm4, power2=power2, pnorm2=pnorm2, color2=color2)

interactive(children=(FloatSlider(value=0.0965754, description='kT1 (keV)', max=0.2, min=0.01, readout_format=…

# Compare two CXB spectra with Three Apec Components

In [18]:
kT = widgets.FloatSlider(value=9.65754E-02,min=0.01,max=0.2,step=0.001, description="kT1 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm = widgets.FloatSlider(value=5.56817E-07,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
kT2 = widgets.FloatSlider(value=0.178613,min=0.01,max=0.7,step=0.001, description="kT2 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm2 = widgets.FloatSlider(value=1.64881E-06,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
kT3 = widgets.FloatSlider(value=0.635963,min=0.01,max=0.9,step=0.001, description="kT3 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm3 = widgets.FloatSlider(value=8.85404E-08,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
nH = widgets.FloatSlider(value=0.084,min=0.01,max=0.1,step=0.001, description="Photoelectric Absorpsion",readout=True,readout_format='.5f')
power = widgets.FloatSlider(value=1.41,min=0,max=3,step=0.1, description="PhoIndex",readout=True,readout_format='.5f')
pnorm = widgets.FloatSlider(value=7.80479E-07,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
color = widgets.Dropdown(options=[('Green',seaborngreen),('Red',seabornred),('Blue',seabornblue),('Orange',seabornorange),('Pink',seabornpink),('Brown',seabornbrown),('Purple',seabornpurple),('Cyan',seaborncyan),('Magenta',seabornmagenta),('Yellow',seabornyellow)], description='Color')

kT4 = widgets.FloatSlider(value=9.65754E-02,min=0.01,max=0.2,step=0.001, description="kT1 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm4 = widgets.FloatSlider(value=5.56817E-07,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
kT5 = widgets.FloatSlider(value=0.178613,min=0.01,max=0.7,step=0.001, description="kT2 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm5 = widgets.FloatSlider(value=1.64881E-06,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
kT6 = widgets.FloatSlider(value=0.635963,min=0.01,max=0.9,step=0.001, description="kT3 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm6 = widgets.FloatSlider(value=8.85404E-08,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
nH2 = widgets.FloatSlider(value=0.084,min=0.01,max=0.1,step=0.001, description="Photoelectric Absorpsion",readout=True,readout_format='.5f')
power2 = widgets.FloatSlider(value=1.41,min=0,max=3,step=0.1, description="PhoIndex",readout=True,readout_format='.5f')
pnorm2 = widgets.FloatSlider(value=7.80479E-07,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
color2 = widgets.Dropdown(options=[('Green',seaborngreen),('Red',seabornred),('Blue',seabornblue),('Orange',seabornorange),('Pink',seabornpink),('Brown',seabornbrown),('Purple',seabornpurple),('Cyan',seaborncyan),('Magenta',seabornmagenta),('Yellow',seabornyellow)], description='Color')


def cxb_model_3apec(kT, norm, nH, kT2, norm2, kT3, norm3, power, pnorm):
    cxb3 = Model("apec + phabs(apec + apec+ powerlaw)", "cxb3")
    cxb3.apec.kT.values = kT, -1
    cxb3.apec.Abundanc.values = 1, -1
    cxb3.apec.Redshift.values = 0, -1
    cxb3.apec.norm.values =  norm, -1
    cxb3.phabs.nH.values = nH, -1
    cxb3.apec_3.kT.values = kT2, -1
    cxb3.apec_3.Abundanc.values = 1, -1
    cxb3.apec_3.Redshift.values = 0, -1
    cxb3.apec_3.norm.values = norm2, -1
    cxb3.apec_4.kT.values = kT3, -1
    cxb3.apec_4.Abundanc.values = 1, -1
    cxb3.apec_4.Redshift.values = 0, -1
    cxb3.apec_4.norm.values = norm3, -1
    cxb3.powerlaw.PhoIndex.values = power, -1
    cxb3.powerlaw.norm.values = pnorm, -1
    Xset.chatter=0
    Plot.addCommand("rescale x 0.1 10")
    Plot.addCommand("rescale y 1e-14 1e-5")
    Plot("model cxb3")
    mx = Plot.x()
    my = Plot.model()
    AllModels.show()
    return mx,my

def plot_3apec_cxb_models(kT, norm, nH, kT2, norm2, kT3, norm3, power, pnorm, color, kT4, norm4, nH2, kT5, norm5, kT6, norm6, power2, pnorm2, color2):
    fig, ax = plt.subplots(2,figsize=(10,6))
    x,y = cxb_model_3apec(kT, norm, nH, kT2, norm2, kT3, norm3, power, pnorm)
    ax[0].plot(x, y, color=color, label=f'CXB1',linewidth=1)
    x2,y2 = cxb_model_3apec(kT4, norm4, nH2, kT5, norm5, kT6, norm6, power2, pnorm2)
    ax[0].plot(x2, y2, color=color2, label=f'CXB2',linewidth=1)
    ratio = [y/y2 if y2 else 0 for y,y2 in zip(y,y2)]
    #ax.plot(x, y)
    ax[1].set_xlabel('Energy (keV)')
    ax[0].set_ylabel(r'counts/s/keV')
    ax[0].set_xscale("log")
    ax[0].set_ylim((1e-10,1e-2))
    ax[0].set_yscale("log")
    #ax[0].set_xlim((0.5,3))
    ax[0].grid()
    ax[0].legend()
    ax[1].plot(x, ratio, color="black", label=f'M1/M2 ratio',linewidth=1)
    ax[1].set_ylabel(r'Ratio')
    ax[1].set_xscale("log")
    #ax[1].set_ylim((0,5))
    ax[1].set_yscale("linear")
    #ax[1].set_xlim((0.5,3))
    ax[1].grid()
    ax[1].legend()
    plt.show()

#out1 = widgets.interactive_output(plot_apec_model, {'kT': kT, 'Abundanc': Abundanc, 'Redshift': Redshift, 'norm': norm, 'color': color})
#display(out1, ui1)

interactive(plot_3apec_cxb_models, kT=kT, norm=norm, nH=nH, kT2=kT2, norm2=norm2, kT3=kT3, norm3=norm3, power=power, pnorm=pnorm, color=color, kT4=kT4, norm4=norm4, nH2=nH2, kT5=kT5, norm5=norm5, kT6=kT6, norm6=norm6, power2=power2, pnorm2=pnorm2, color2=color2)

interactive(children=(FloatSlider(value=0.0965754, description='kT1 (keV)', max=0.2, min=0.01, readout_format=…

# Compare 2Apec and 3Apec CXB Models

In [5]:
kT = widgets.FloatSlider(value=9.65754E-02,min=0.01,max=0.2,step=0.001, description="kT1 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm = widgets.FloatSlider(value=5.56817E-07,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
kT2 = widgets.FloatSlider(value=0.178613,min=0.01,max=0.7,step=0.001, description="kT2 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm2 = widgets.FloatSlider(value=1.64881E-06,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
kT3 = widgets.FloatSlider(value=0.635963,min=0.01,max=0.9,step=0.001, description="kT3 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm3 = widgets.FloatSlider(value=8.85404E-08,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
nH = widgets.FloatSlider(value=0.084,min=0.01,max=0.1,step=0.001, description="Photoelectric Absorpsion",readout=True,readout_format='.5f')
power = widgets.FloatSlider(value=1.41,min=0,max=3,step=0.1, description="PhoIndex",readout=True,readout_format='.5f')
pnorm = widgets.FloatSlider(value=7.80479E-07,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
color = widgets.Dropdown(options=[('Green',seaborngreen),('Red',seabornred),('Blue',seabornblue),('Orange',seabornorange),('Pink',seabornpink),('Brown',seabornbrown),('Purple',seabornpurple),('Cyan',seaborncyan),('Magenta',seabornmagenta),('Yellow',seabornyellow)], description='Color')

kT4 = widgets.FloatSlider(value=9.65754E-02,min=0.01,max=0.2,step=0.001, description="kT1 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm4 = widgets.FloatSlider(value=5.56817E-07,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
kT5 = widgets.FloatSlider(value=0.178613,min=0.01,max=0.7,step=0.001, description="kT2 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm5 = widgets.FloatSlider(value=1.64881E-06,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
kT6 = widgets.FloatSlider(value=0.635963,min=0.01,max=0.9,step=0.001, description="kT3 (keV)",readout=True,readout_format='.5f')#round((10-0.008)/50,7)
norm6 = widgets.FloatSlider(value=8.85404E-08,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
nH2 = widgets.FloatSlider(value=0.084,min=0.01,max=0.1,step=0.001, description="Photoelectric Absorpsion",readout=True,readout_format='.5f')
power2 = widgets.FloatSlider(value=1.41,min=0,max=3,step=0.1, description="PhoIndex",readout=True,readout_format='.5f')
pnorm2 = widgets.FloatSlider(value=7.80479E-07,min=1e-10,max=1e-4,step=round((1e-4-1e-10)/10000,7), description="Normalization",readout=True,readout_format='.4e')
color2 = widgets.Dropdown(options=[('Green',seaborngreen),('Red',seabornred),('Blue',seabornblue),('Orange',seabornorange),('Pink',seabornpink),('Brown',seabornbrown),('Purple',seabornpurple),('Cyan',seaborncyan),('Magenta',seabornmagenta),('Yellow',seabornyellow)], description='Color')



def cxb_model_2apec(kT, norm, nH, kT2, norm2, power, pnorm):
    cxb2 = Model("apec + phabs(apec + powerlaw)", "cxb2")
    cxb2.apec.kT.values = kT, -1
    cxb2.apec.Abundanc.values = 1, -1
    cxb2.apec.Redshift.values = 0, -1
    cxb2.apec.norm.values =  norm, -1
    cxb2.phabs.nH.values = nH, -1
    cxb2.apec_3.kT.values = kT2, -1
    cxb2.apec_3.Abundanc.values = 1, -1
    cxb2.apec_3.Redshift.values = 0, -1
    cxb2.apec_3.norm.values = norm2, -1
    cxb2.powerlaw.PhoIndex.values = power, -1
    cxb2.powerlaw.norm.values = pnorm, -1
    Xset.chatter=0
    Plot.addCommand("rescale x 0.1 10")
    Plot.addCommand("rescale y 1e-14 1e-5")
    Plot("model cxb2")
    mx = Plot.x()
    my = Plot.model()
    AllModels.show()
    return mx,my

def cxb_model_3apec(kT, norm, nH, kT2, norm2, kT3, norm3, power, pnorm):
    cxb3 = Model("apec + phabs(apec + apec+ powerlaw)", "cxb3")
    cxb3.apec.kT.values = kT, -1
    cxb3.apec.Abundanc.values = 1, -1
    cxb3.apec.Redshift.values = 0, -1
    cxb3.apec.norm.values =  norm, -1
    cxb3.phabs.nH.values = nH, -1
    cxb3.apec_3.kT.values = kT2, -1
    cxb3.apec_3.Abundanc.values = 1, -1
    cxb3.apec_3.Redshift.values = 0, -1
    cxb3.apec_3.norm.values = norm2, -1
    cxb3.apec_4.kT.values = kT3, -1
    cxb3.apec_4.Abundanc.values = 1, -1
    cxb3.apec_4.Redshift.values = 0, -1
    cxb3.apec_4.norm.values = norm3, -1
    cxb3.powerlaw.PhoIndex.values = power, -1
    cxb3.powerlaw.norm.values = pnorm, -1
    Xset.chatter=0
    Plot.addCommand("rescale x 0.1 10")
    Plot.addCommand("rescale y 1e-14 1e-5")
    Plot("model cxb3")
    mx = Plot.x()
    my = Plot.model()
    AllModels.show()
    return mx,my

def plot_2_vs_3apec_cxb_models(kT, norm, nH, kT2, norm2, power, pnorm, color, kT4, norm4, nH2, kT5, norm5, kT6, norm6, power2, pnorm2, color2):
    fig, ax = plt.subplots(2,figsize=(10,6))
    x,y = cxb_model_2apec(kT, norm, nH, kT2, norm2, power, pnorm)
    ax[0].plot(x, y, color=color, label=f'2Apec CXB',linewidth=1)
    x2,y2 = cxb_model_3apec(kT4, norm4, nH2, kT5, norm5, kT6, norm6, power2, pnorm2)
    ax[0].plot(x2, y2, color=color2, label=f'3Apec CXB',linewidth=1)
    ratio = [y/y2 if y2 else 0 for y,y2 in zip(y,y2)]
    #ax.plot(x, y)
    ax[1].set_xlabel('Energy (keV)')
    ax[0].set_ylabel(r'counts/s/keV')
    ax[0].set_xscale("log")
    ax[0].set_ylim((1e-10,1e-2))
    ax[0].set_yscale("log")
    #ax[0].set_xlim((0.5,3))
    ax[0].grid()
    ax[0].legend()
    ax[1].plot(x, ratio, color="black", label=f'M1/M2 ratio',linewidth=1)
    ax[1].set_ylabel(r'Ratio')
    ax[1].set_xscale("log")
    #ax[1].set_ylim((0,5))
    ax[1].set_yscale("linear")
    #ax[1].set_xlim((0.5,3))
    ax[1].grid()
    ax[1].legend()
    plt.show()

#out1 = widgets.interactive_output(plot_apec_model, {'kT': kT, 'Abundanc': Abundanc, 'Redshift': Redshift, 'norm': norm, 'color': color})
#display(out1, ui1)

interactive(plot_2_vs_3apec_cxb_models, kT=kT, norm=norm, nH=nH, kT2=kT2, norm2=norm2, power=power, pnorm=pnorm, color=color, kT4=kT4, norm4=norm4, nH2=nH2, kT5=kT5, norm5=norm5, kT6=kT6, norm6=norm6, power2=power2, pnorm2=pnorm2, color2=color2)

interactive(children=(FloatSlider(value=0.0965754, description='kT1 (keV)', max=0.2, min=0.01, readout_format=…